## Introduction

In this notebook we try to study the data of CO2 emmissions in Europe and try to answer some pressing questions. 
//TODO: Explain the dataset data and explain why the dataset was chosen.

## Step 1. Get the data
The data are stored into a csv file called owid-co-2data . This file contains the information of CO2 emissions for every (most) country in the world. There are also aggregated data for the continents.

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns


df = pd.read_csv('/kaggle/input/d/darkdustd/co2-data/owid-co2-data.csv')

### Step 2. Explore the data

In [ ]:
total_emissions = df.groupby('country')['co2'].sum().reset_index()

# Sort the countries by total CO2 emissions in descending order
total_emissions = total_emissions.sort_values(by='co2', ascending=False).reset_index(drop=True)

# Display the top 10 emitting countries
top_emitters = total_emissions.head(10)


plt.figure(figsize=(10, 6))
plt.bar(top_emitters['country'], top_emitters['co2'], color='blue')
plt.title('Top 10 emitting countries', fontsize=14)
plt.xlabel('Country', fontsize=12)
plt.ylabel('Total CO2 Emissions (million tonnes)', fontsize=12)
plt.xticks(rotation=45, ha='right', fontsize=10)
plt.tight_layout()
plt.show()

We see that we also have data related to the continents and other entities or groups. Since we're focusing on Europe , the Europe data will be very useful.

### How is the level of carbon emission in Europe evolving?
As mentioned before , we only need the data for the countries of Europe.As we saw above, in our data we see that we have in the country column aggregates for each continent. In our case we need the ones for Europe. We'll use the 'year' and the 'co2' column to create a plot with matplotlib.

In [ ]:

europe_aggregate_data = df[df['country'] == 'Europe']

# Dropping years that have no data
europe_emission_trend_aggregate = europe_aggregate_data[['year', 'co2']].dropna()

# Plot the trend for the aggregate data
plt.figure(figsize=(10, 6))
plt.plot(europe_emission_trend_aggregate['year'], europe_emission_trend_aggregate['co2'], marker='o', linestyle='-')
plt.title('Evolution of CO2 Emissions in Europe', fontsize=14)
plt.xlabel('Year', fontsize=12)
plt.ylabel('Total CO2 Emissions (in million tonnes)', fontsize=12)
plt.grid(True)
plt.tight_layout()
plt.show()



To see that the data is the same , we can manually aggregate the data and we expect to see the same result.

In [ ]:
european_countries = [
    "Albania", "Andorra", "Armenia", "Austria", "Azerbaijan", "Belarus", "Belgium",
    "Bosnia and Herzegovina", "Bulgaria", "Croatia", "Cyprus", "Czechia", "Denmark",
    "Estonia", "Finland", "France", "Georgia", "Germany", "Greece", "Hungary", "Iceland",
    "Ireland", "Italy", "Kazakhstan", "Kosovo", "Latvia", "Liechtenstein", "Lithuania",
    "Luxembourg", "Malta", "Moldova", "Monaco", "Montenegro", "Netherlands", "North Macedonia",
    "Norway", "Poland", "Portugal", "Romania", "Russia", "San Marino", "Serbia",
    "Slovakia", "Slovenia", "Spain", "Sweden", "Switzerland", "Turkey", "Ukraine",
    "United Kingdom", "Vatican City"
]

# Filter the dataset for European countries
europe_data = df[df['country'].isin(european_countries)]

europe_emission_trend = europe_data.groupby('year')['co2'].sum().reset_index()

# Plot the trend
plt.figure(figsize=(10, 6))
plt.plot(europe_emission_trend['year'], europe_emission_trend['co2'], marker='o', linestyle='-')
plt.title('Evolution of CO2 Emissions in Europe (calculated by country)', fontsize=14)
plt.xlabel('Year', fontsize=12)
plt.ylabel('Total CO2 Emissions (in million tonnes)', fontsize=12)
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Calculate total emissions by country in Europe
europe_total_emissions = europe_data.groupby('country')['co2'].sum().reset_index()
europe_total_emissions = europe_total_emissions.sort_values(by='co2', ascending=False).reset_index(drop=True)

# Get the top 10 countries by emissions
top_10_countries = europe_total_emissions['country'].head(10)

# Filter the data for only the top 10 countries
top_10_data = europe_data[europe_data['country'].isin(top_10_countries)]

# Prepare data for stacked area chart
top_10_trend = top_10_data.pivot_table(index='year', columns='country', values='co2', aggfunc='sum').fillna(0)

# Plot the stacked area chart
top_10_trend.plot(kind='area', stacked=True, figsize=(12, 8), alpha=0.7)
plt.title('CO2 Emissions by Top 10 European Countries Over Time', fontsize=14)
plt.xlabel('Year', fontsize=12)
plt.ylabel('CO2 Emissions (million tonnes)', fontsize=12)
plt.legend(loc='upper left', fontsize=10, title='Country')
plt.tight_layout()
plt.show()

We see a sharp increase in the middle of 20th century. The expansion of manufacturing industries and the widespread adoption of fossil fuels, particularly coal and oil,after WW2,probably drove this increase. After the 1990s gradual decline in emissions is apparent.European countries began adopting strict environmental policies and EU-wide regulations to reduce greenhouse gases.

### Which countries are emitting and 
absorbing the most carbon?

In [ ]:
# Calculate total emissions by country in Europe
europe_emissions = europe_data.groupby('country')['co2'].sum().reset_index()
europe_emissions = europe_emissions.sort_values(by='co2', ascending=False).reset_index(drop=True)

# Calculate total absorption by country (land_use_change_co2) in Europe
europe_absorption = europe_data.groupby('country')['land_use_change_co2'].sum().reset_index()
europe_absorption = europe_absorption[europe_absorption['land_use_change_co2'] < 0] # keep only countries with negative values 
europe_absorption = europe_absorption.sort_values(by='land_use_change_co2').reset_index(drop=True)

# Bar chart visualization for emissions
plt.figure(figsize=(10, 6))
plt.bar(europe_emissions.head(5)['country'], europe_emissions.head(5)['co2'], color='blue')
plt.title('Top 5 CO2 Emitting Countries (Europe)', fontsize=14)
plt.xlabel('Country', fontsize=12)
plt.ylabel('Total CO2 Emissions (million tonnes)', fontsize=12)
plt.xticks(rotation=45, ha='right', fontsize=10)
plt.tight_layout()
plt.show()

# Bar chart visualization for absorbers
plt.figure(figsize=(10, 6))
plt.bar(europe_absorption.head(5)['country'], europe_absorption.head(5)['land_use_change_co2'], color='blue')
plt.title('Top 5 CO2 Absorbing Countries (Europe)', fontsize=14)
plt.xlabel('Country', fontsize=12)
plt.ylabel('Total CO2 Emissions (million tonnes)', fontsize=12)
plt.xticks(rotation=45, ha='right', fontsize=10)
plt.tight_layout()
plt.show()

We see that larger countries or wealthier nations are at the top of the table for emitting CO2. However , most of these countries are also at the top of the absorbing list. We can do a similar thing for 'co2_per_capita' and 'co2_per_gdp' , and see if we find any differences. 

In [ ]:
# Drop rows with missing data for `co2_per_capita` or `co2_per_gdp`
europe_data = europe_data.dropna(subset=['co2_per_capita', 'co2_per_gdp'])

# Aggregate to find average values per country
europe_co2_stats = europe_data.groupby('country').agg({
    'co2': 'sum',
    'population': 'mean',
    'gdp': 'mean',
    'co2_per_capita': 'mean',
    'co2_per_gdp': 'mean'
}).reset_index()

# Sort by CO2 per capita and per GDP
sorted_by_per_capita = europe_co2_stats.sort_values(by='co2_per_capita', ascending=False)
sorted_by_per_gdp = europe_co2_stats.sort_values(by='co2_per_gdp', ascending=False)

# Bar chart visualization for emitting countries by population
plt.figure(figsize=(10, 6))
plt.bar(sorted_by_per_capita.head(5)['country'], sorted_by_per_capita.head(5)['co2_per_capita'], color='blue')
plt.title('Top 5 CO2 Emitting Countries in Europe (Per Capita)', fontsize=14)
plt.xlabel('Country', fontsize=12)
plt.ylabel('Total CO2 Emissions (million tonnes)', fontsize=12)
plt.xticks(rotation=45, ha='right', fontsize=10)
plt.tight_layout()
plt.show()

plt.figure(figsize=(10, 6))
plt.bar(sorted_by_per_capita.tail(5)['country'], sorted_by_per_capita.tail(5)['co2_per_capita'], color='blue')
plt.title('Top 5 lowest CO2 Emitting Countries in Europe (Per Capita)', fontsize=14)
plt.xlabel('Country', fontsize=12)
plt.ylabel('Total CO2 Emissions (million tonnes)', fontsize=12)
plt.xticks(rotation=45, ha='right', fontsize=10)
plt.tight_layout()
plt.show()

# Bar chart visualization for emitting countries by GDP
plt.figure(figsize=(10, 6))
plt.bar(sorted_by_per_gdp.head(5)['country'], sorted_by_per_gdp.head(5)['co2_per_gdp'], color='blue')
plt.title('Top 5 CO2 Emitting Countries in Europe (Per GDP)', fontsize=14)
plt.xlabel('Country', fontsize=12)
plt.ylabel('Total CO2 Emissions (million tonnes)', fontsize=12)
plt.xticks(rotation=45, ha='right', fontsize=10)
plt.tight_layout()
plt.show()

plt.figure(figsize=(10, 6))
plt.bar(sorted_by_per_gdp.tail(5)['country'], sorted_by_per_gdp.tail(5)['co2_per_gdp'], color='blue')
plt.title('Top 5 lowest CO2 Emitting Countries in Europe (Per GDP)', fontsize=14)
plt.xlabel('Country', fontsize=12)
plt.ylabel('Total CO2 Emissions (million tonnes)', fontsize=12)
plt.xticks(rotation=45, ha='right', fontsize=10)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(df['population'], df['co2'], c=df['land_use_change_co2'], cmap='coolwarm', alpha=0.7)
plt.title('CO2 Emissions vs. Population (with Absorption Levels)', fontsize=14)
plt.xlabel('Population', fontsize=12)
plt.ylabel('Total CO2 Emissions (million tonnes)', fontsize=12)
plt.colorbar(label='Land Use Change CO2')
plt.grid(True)
plt.tight_layout()
plt.show()

The 'co2_per_capita' measures the amount of CO2 emissions produced per individual in a country.As expected countries with low population tend to rank high in this rankings. Ukraine and Turkey , for example, have relatively low per capita emissions due to their larger populations, even if total emissions are high.**

In [ ]:
# Combine emitters and absorbers into one dataset
europe_emissions['type'] = 'Emitter'
europe_absorption['type'] = 'Absorber'
comparison = pd.concat([europe_emissions, europe_absorption.rename(columns={'land_use_change_co2': 'co2'})])

# Stacked bar chart
plt.figure(figsize=(12, 6))
for label, group in comparison.groupby('type'):
    plt.bar(group['country'], group['co2'], label=label)
plt.title('Emitters vs Absorbers (Stacked)', fontsize=14)
plt.xlabel('Country', fontsize=12)
plt.ylabel('CO2 (million tonnes)', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.legend()
plt.tight_layout()
plt.show()

## Are there differences when considering all carbon ever emitted
by a country?

//TODO Fjona : Study data and build graphs to try and answer the question. 

## Are there characteristics of countries that correlate with the changes in the carbon emissions

//TODO ,Nicolas & Khasrur: Build and train a model if necessary. ?

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
# Attributes and target variable
attributes = [
    "population", # Population of the country
    "gdp", # Gross Domestic Product of the country
    "primary_energy_consumption", # Primary energy consumption
    "cement_co2", # Annual CO₂ emissions from cement
    "cement_co2_per_capita", # CO₂ emissions from cement per capita
    "coal_co2", # Annual CO₂ emissions from coal
    "coal_co2_per_capita", # CO₂ emissions from coal per capita
    "land_use_change_co2", # CO₂ emissions from land use change
    "land_use_change_co2_per_capita", # CO₂ emissions from land use change per capita
    "methane", # Annual methane emissions
    "methane_per_capita", # Methane emissions per capita
    "oil_co2", # Annual CO₂ emissions from oil
    "oil_co2_per_capita", # CO₂ emissions from oil per capita    
]
target = "co2"

# Copy europe data
model_data = europe_data.copy()

# Keep only the attributes and target
model_data = model_data[attributes + [target]]

## Correlation between picked attributes

In [ ]:
# Correlation matrix for europe_data
plt.figure(figsize=(10, 8))
sns.heatmap(model_data.corr(), annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix for Europe Data', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
attributes_to_drop = [
    "primary_energy_consumption",
    "methane"
]

# Remove highly correlated features
model_data_2 = model_data.drop(columns=attributes_to_drop)

# Remove attibutes from the list
attributes = [attr for attr in attributes if attr not in attributes_to_drop]

In [ ]:
# Drop rows with missing target
model_data_2 = model_data_2.dropna(subset=[target])

# Fill missing attributes with mean of the 2 years in between
model_data_2[attributes] = model_data_2[attributes].interpolate(limit_direction='both')

X = model_data_2[attributes]
y = model_data_2[target]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Min-max scaling using sklearn
scaler = MinMaxScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
best_model = RandomForestRegressor(
    n_estimators=200,
    min_samples_split=2,
    min_samples_leaf=1,
    max_depth=None,
    random_state=42
)

# Fit the model
best_model.fit(X_train_scaled, y_train)


In [ ]:
# Predictions
y_pred = best_model.predict(X_test_scaled)

# Evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print(f"MAE: {mae}")
print(f"MSE: {mse}")
print(f"RMSE: {rmse}")

In [ ]:
# Get feature importances
importances = best_model.feature_importances_
indices = np.argsort(importances)[::-1]

plt.figure(figsize=(10, 6))
plt.bar(range(len(attributes)), importances[indices], align='center')
plt.xticks(range(len(attributes)), [attributes[i] for i in indices], rotation=90)
plt.title('Feature Importances', fontsize=14)
plt.show()

In [ ]:
# Predictions vs Actual plot
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, c='blue', alpha=0.7)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2)
plt.xlabel('Actual CO2 Emissions (million tonnes)', fontsize=12)
plt.ylabel('Predicted CO2 Emissions (million tonnes)', fontsize=12)
plt.title('Actual vs Predicted CO2 Emissions', fontsize=14)
plt.grid(True)
plt.show()

### Population vs CO2 emissions

In [ ]:
population_millions = europe_data['population']
co2_emissions = europe_data['co2']

plt.figure(figsize=(10, 6))
plt.scatter(population_millions, co2_emissions)
plt.title('Population vs CO2 Emissions', fontsize=14)
plt.xlabel('Population Avg (Millions)', fontsize=12)
plt.ylabel('CO2 Emissions (million tonnes)', fontsize=12)
plt.grid(True)
plt.show()

In [ ]:
# Population in millions by country 
population_millions = europe_data.groupby('country')['population'].mean()
co2_emissions = europe_data.groupby('country')['co2'].sum()

plt.figure(figsize=(10, 6))
plt.scatter(population_millions, co2_emissions)
plt.title('Population vs CO2 Emissions', fontsize=14)
plt.xlabel('Population Avg (Millions)', fontsize=12)
plt.ylabel('CO2 Emissions (million tonnes)', fontsize=12)
plt.grid(True)
plt.show()

In [ ]:
combined_population_vs_co2_data = pd.DataFrame({
    "population_millions": population_millions,
    "co2_emissions": co2_emissions
})

sorted_data_by_co2 = combined_population_vs_co2_data.sort_values(by="co2_emissions", ascending=False)

print(sorted_data_by_co2[:5])

sorted_data_by_population = combined_population_vs_co2_data.sort_values(by="population_millions", ascending=False)

print(sorted_data_by_population[:5])

## Conclusions
TODO: Talk about 
difficulties in analysing the dat,?wWhat were the key insights obtaine andsw Which Data Science tools and techniques were learned
during this exerc.up?